In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[3], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [8]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 10s 178ms/step - loss: 0.6802 - accuracy: 0.5575
Epoch 2/30
58/58 [==============================] - 10s 179ms/step - loss: 0.5699 - accuracy: 0.6922
Epoch 3/30
58/58 [==============================] - 10s 179ms/step - loss: 0.5219 - accuracy: 0.7536
Epoch 4/30
58/58 [==============================] - 10s 178ms/step - loss: 0.4327 - accuracy: 0.8248
Epoch 5/30
58/58 [==============================] - 10s 179ms/step - loss: 0.4841 - accuracy: 0.8028
Epoch 6/30
58/58 [==============================] - 10s 179ms/step - loss: 0.4336 - accuracy: 0.8302
Epoch 7/30
58/58 [==============================] - 10s 180ms/step - loss: 0.4392 - accuracy: 0.8193
Epoch 8/30
58/58 [==============================] - 10s 179ms/step - loss: 0.4224 - accuracy: 0.8127
Epoch 9/30
58/58 [==============================] - 10s 179ms/step - loss: 0.3941 - accuracy: 0.8401
Epoch 10/30
58/58 [==============================] - 10s 176ms/step - loss: 0.3811 - accura

58/58 [==============================] - 11s 184ms/step - loss: 0.3573 - accuracy: 0.8478
Epoch 22/30
58/58 [==============================] - 11s 184ms/step - loss: 0.3231 - accuracy: 0.8697
Epoch 23/30
58/58 [==============================] - 10s 181ms/step - loss: 0.2594 - accuracy: 0.9014
Epoch 24/30
58/58 [==============================] - 10s 179ms/step - loss: 0.2353 - accuracy: 0.9102
Epoch 25/30
58/58 [==============================] - 10s 178ms/step - loss: 0.2432 - accuracy: 0.9091
Epoch 26/30
58/58 [==============================] - 10s 179ms/step - loss: 0.2415 - accuracy: 0.9102
Epoch 27/30
58/58 [==============================] - 11s 181ms/step - loss: 0.2850 - accuracy: 0.8839
Epoch 28/30
58/58 [==============================] - 11s 185ms/step - loss: 0.2469 - accuracy: 0.9124
Epoch 29/30
58/58 [==============================] - 10s 180ms/step - loss: 0.1699 - accuracy: 0.9409
Epoch 30/30
58/58 [==============================] - 11s 185ms/step - loss: 0.1518 - accuracy:

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 19s 330ms/step - loss: 0.6826 - accuracy: 0.5520
Epoch 2/30
58/58 [==============================] - 19s 325ms/step - loss: 0.5675 - accuracy: 0.7141
Epoch 3/30
58/58 [==============================] - 19s 329ms/step - loss: 0.4858 - accuracy: 0.7864
Epoch 4/30
58/58 [==============================] - 19s 330ms/step - loss: 0.5142 - accuracy: 0.7809
Epoch 5/30
58/58 [==============================] - 19s 325ms/step - loss: 0.5594 - accuracy: 0.7262
Epoch 6/30
58/58 [==============================] - 19s 329ms/step - loss: 0.5374 - accuracy: 0.7645
Epoch 7/30
58/58 [==============================] - 19s 325ms/step - loss: 0.3985 - accuracy: 0.8467
Epoch 8/30
58/58 [==============================] - 19s 326ms/step - loss: 0.4000 - accuracy: 0.8445
Epoch 9/30
58/58 [==============================] - 19s 327ms/step - loss: 0.3649 - accuracy: 0.8642
Epoch 10/30
58/58 [==============================] - 19s 325ms/step - loss: 0.3377 - accura

58/58 [==============================] - 18s 317ms/step - loss: 0.1813 - accuracy: 0.9430
Epoch 22/30
58/58 [==============================] - 17s 301ms/step - loss: 0.3317 - accuracy: 0.8664
Epoch 23/30
58/58 [==============================] - 17s 286ms/step - loss: 0.3217 - accuracy: 0.8970
Epoch 24/30
58/58 [==============================] - 17s 290ms/step - loss: 0.2973 - accuracy: 0.8740
Epoch 25/30
58/58 [==============================] - 17s 299ms/step - loss: 0.1761 - accuracy: 0.9474
Epoch 26/30
58/58 [==============================] - 17s 293ms/step - loss: 0.1552 - accuracy: 0.9409
Epoch 27/30
58/58 [==============================] - 17s 292ms/step - loss: 0.1148 - accuracy: 0.9617
Epoch 28/30
58/58 [==============================] - 17s 292ms/step - loss: 0.0850 - accuracy: 0.9792
Epoch 29/30
58/58 [==============================] - 17s 290ms/step - loss: 0.0647 - accuracy: 0.9858
Epoch 30/30
58/58 [==============================] - 17s 291ms/step - loss: 0.1698 - accuracy: